In [1]:
!pip install transformers sentencepiece


In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Choose model
model_name = "t5-small"  #'google/flan-t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
# TESTING

import time

job_ad = "We are looking for a software developer to join our team in our downtown office in New York. Must be available to work in person 5 days a week."

# Format the prompt

prompt = f"Classify the work arrangement of the following job ad: {job_ad}"

start_time = time.time()
# Tokenize
input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device) # Move input_ids to the same device as the model

# Generate output
job_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

stop_time = time.time()
execution_time = stop_time - start_time
print("Execution time:", execution_time, "seconds")

# Decode and print
print("Summary:", tokenizer.decode(job_ids[0], skip_special_tokens=True))

Execution time: 1.2299306392669678 seconds
Summary: . Must be available to work in person 5 days a week.


In [4]:
import os
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/AB_job_data_files'

import os
from google.colab import drive

# find location of different files:
file_location = {}
# Check if the folder exists
if os.path.exists(folder_path):
  os.chdir(folder_path)
  for num, f in enumerate(os.listdir()):
    file_location[f] = os.path.join(folder_path, f)

else:
  print(f"Folder not found: {folder_path}")

print(file_location)

Mounted at /content/drive
{'seniority_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_test_set.csv', 'unlabelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/unlabelled_development_set.csv', 'salary_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_development_set.csv', 'seniority_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_development_set.csv', 'work_arrangements_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_test_set.csv', 'salary_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_test_set.csv', 'work_arrangements_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_development_set.csv', 'results': '/content/drive/MyDrive/AB_job_data_files/results', 'wandb': '/content/drive/MyDrive/AB_job_data_files/wandb', 't5-small-work-arrangement-finetuned': '/content/d

In [5]:
def work_arrang(job_ad):

    # Format the prompt
    prompt = f"Classify the work arrangement of the following job ad as one of the following: on-site, Remote, or Hybrid {job_ad}"

    # Tokenize
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device) # Move input_ids to the same device as the model

    # Generate output
    job_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

    # Decode and return
    return tokenizer.decode(job_ids[0], skip_special_tokens=True)

In [10]:
# DEV set
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

work_arrangement_dev = pd.read_csv(file_location['work_arrangements_development_set.csv'])

results = 0
count = 0

# Get labels and convert to lowercase for consistency
true_labels = [label.lower() for label in work_arrangement_dev['y_true']]

# Make predictions
predictions = []

for index, row in work_arrangement_dev.iterrows():
    predicted_label = work_arrang(row['job_ad']).lower()
    if predicted_label == "on-site":
        predicted_label = "onsite"

    true_label =  row['y_true'].lower()

    print(predicted_label, '----', true_label)
    print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

    # Append the prediction to the predictions list
    predictions.append(predicted_label)

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)

# calculate metrics
accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

# print results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)
print("\nConfusion Matrix:")
print(conf_matrix)



,,,,, and your natural approach is inclusive, respectful and professional. please complete a cover letter, (up to 3 pages) outlining this and send with your resume to marita ryan at ---- remote
False
: home-based online esl teacher (online english teacher) we look for passionate and skilled teachers for both full-time and part time who would like to join our growing tutor community! employer: et-phone location: ---- remote
False
: senior, future-focused role, location: chullora, sydney, commencement: by january 2025. are you seeking a safeguarding position which is: a mission-based, senior role with a profound ---- hybrid
False
,,,,.:: on-site, remote, or hybrid job title: delivery driver abstract: pickup then deliver food or other items to customers. choose when, how, and where ---- onsite
False
specsavers australia's optical market leader. apply now! *please note due to the high volume of applications only successful candidates will be contacted. ---- onsite
False
, wa600 front end l

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [11]:
# TEST set
import pandas as pd

work_arrangement_test = pd.read_csv(file_location['work_arrangements_test_set.csv'])

results = 0
count = 0

# Get labels and convert to lowercase for consistency
true_labels = [label.lower() for label in work_arrangement_dev['y_true']]

# Make predictions
predictions = []

for index, row in work_arrangement_test.iterrows():
    predicted_label = work_arrang(row['job_ad']).lower()
    if predicted_label == "on-site":
        predicted_label = "onsite"

    true_label =  row['y_true'].lower()

    #print(work_arrang(row['job_ad']), '----', row['y_true'])
    #print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

    # Append the prediction to the predictions list
    predictions.append(predicted_label)

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)

# calculate metrics
accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

# print results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)
print("\nConfusion Matrix:")
print(conf_matrix)




count: 99
result: 0
correctly indentified: 0.0
Accuracy: 0.0000

Classification Report:
                                                                                                                                                                                                                                                                                                          precision    recall  f1-score   support

                                                                                                                                           (open to fresh graduates): on-site, remote, or hybrid job title: finance operations specialist - seamoney credit abstract: hybrid work set-up (once a week wf       0.00      0.00      0.00       0.0
                                                                                                                               , - no weekend work!, full time hours each week (37.5pw). your next role:if you love going the extra mile fo

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_